In [1]:
from google.colab import files, drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets

     |████████████████████████████████| 194kB 6.2MB/s 
     |████████████████████████████████| 112kB 9.7MB/s 
     |████████████████████████████████| 245kB 9.8MB/s 


In [3]:
!pip install transformers

     |████████████████████████████████| 2.0MB 6.4MB/s 
     |████████████████████████████████| 890kB 14.4MB/s 
     |████████████████████████████████| 3.2MB 22.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=d53a86d9962db7e5c29e0363af16770299f152f093f8c60830304629ea8ce3d1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [ ]:
!ls /content/drive/MyDrive/DDI/pharma_bert_training/mod

cleanup_no_redundancy_combined.txt  PhamaBert.ipynb


%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/DDI/pharma_bert_training/cleanup_no_redundancy_combined.txt",
    block_size=128,
)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DDI/pharma_bert_training/model_parameters",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "/content/drive/MyDrive/DDI/pharma_bert_training/cleanup_no_redundancy_combined.txt"})

Using custom data configuration default-b1cbeefea4107e9f


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-b1cbeefea4107e9f/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 128

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [5]:
model_checkpoint = "napsternxg/scibert_scivocab_uncased_ft_mlm_SDU21_AI"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import Trainer, TrainingArguments


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/DDI/pharma_bert_training/model_parameters",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=1,
    prediction_loss_only=True,
)

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    #eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,1.485500
1000,1.228900
1500,1.140000
2000,1.082600
2500,1.041700
3000,1.014400
3500,0.983700
4000,0.960400
4500,0.942400
5000,0.922400


Step,Training Loss
500,1.485500
1000,1.228900
1500,1.140000
2000,1.082600
2500,1.041700
3000,1.014400
3500,0.983700
4000,0.960400
4500,0.942400
5000,0.922400


TrainOutput(global_step=19286, training_loss=0.8721524865303645, metrics={'train_runtime': 18738.5994, 'train_samples_per_second': 1.029, 'total_flos': 1.0422003808762368e+17, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 326266, 'init_mem_gpu_alloc_delta': 441293312, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 785967, 'train_mem_gpu_alloc_delta': 1325256704, 'train_mem_cpu_peaked_delta': 119011728, 'train_mem_gpu_peaked_delta': 11156640768})

In [ ]:
trainer.save_model("/content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_SDU/")

In [6]:
model_checkpoint = "napsternxg/scibert_scivocab_uncased_ft_mlm_SDU21_AI"

In [7]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_SDU/",
    tokenizer=model_checkpoint
)

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_SDU/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
fill_mask_regular = pipeline(
    "fill-mask",
    model=model_checkpoint,
    tokenizer=model_checkpoint
)

Some weights of BertModel were not initialized from the model checkpoint at napsternxg/scibert_scivocab_uncased_ft_mlm_SDU21_AI and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#Diarrhea may be an early symptom of incomplete intestinal obstruction, 
#especially in patients with ileostomy or colostomy.
fill_mask("[MASK] may be an early symptom of incomplete intestinal obstruction,\
especially in patients with ileostomy or colostomy.")

[{'score': 0.7008383870124817,
  'sequence': 'diarrhea may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 16557,
  'token_str': 'diarrhea'},
 {'score': 0.15508608520030975,
  'sequence': 'vomiting may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 18644,
  'token_str': 'vomiting'},
 {'score': 0.05971681699156761,
  'sequence': 'constipation may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 29795,
  'token_str': 'constipation'},
 {'score': 0.022686924785375595,
  'sequence': 'nausea may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 18816,
  'token_str': 'nausea'},
 {'score': 0.015483860857784748,
  'sequence': 'fever may be an early symptom of incomplete intestinal obstruction, especial

In [11]:
#Diarrhea may be an early symptom of incomplete intestinal obstruction, 
#especially in patients with ileostomy or colostomy.
fill_mask_regular("[MASK] may be an early symptom of incomplete intestinal obstruction,\
especially in patients with ileostomy or colostomy.")

[{'score': 0.38035503029823303,
  'sequence': 'it may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 256,
  'token_str': 'it'},
 {'score': 0.1818888932466507,
  'sequence': 'this may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 238,
  'token_str': 'this'},
 {'score': 0.08637246489524841,
  'sequence': 'dic may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 10756,
  'token_str': 'dic'},
 {'score': 0.03182947263121605,
  'sequence': 'cd may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.',
  'token': 1389,
  'token_str': 'cd'},
 {'score': 0.014126553200185299,
  'sequence': 'regurgitation may be an early symptom of incomplete intestinal obstruction, especially in patients with ileostomy or colostomy.'

#Abdominal pain, gastrointestinal disturbances, gram-negative folliculitis, \
#eye pain and contact dermatitis have also been reported in association with \
#the use of topical formulations of clindamycin.

fill_mask("[MASK] pain, gastrointestinal disturbances, gram-negative \
folliculitis, eye pain and contact dermatitis have also been reported in \
association with the use of topical formulations of clindamycin.")

#Abdominal pain, gastrointestinal disturbances, gram-negative folliculitis, \
#eye pain and contact dermatitis have also been reported in association with \
#the use of topical formulations of clindamycin.

fill_mask_regular("[MASK] pain, gastrointestinal disturbances, gram-negative \
folliculitis, eye pain and contact dermatitis have also been reported in \
association with the use of topical formulations of clindamycin.")

In [15]:
#AYGESTIN (norethindrone acetate tablets USP) is indicated for the treatment of\ 
#secondary amenorrhea, [endometriosis], and abnormal uterine bleeding due to \
#hormonal imbalance in the absence of organic pathology, such as submucous 
#fibroids or uterine cancer.

fill_mask("AYGESTIN (norethindrone acetate tablets USP) is indicated for the \
treatment of secondary amenorrhea, [MASK], and abnormal uterine bleeding due to\
 hormonal imbalance in the absence of organic pathology, such as submucous \
 fibroids or uterine cancer.")

[{'score': 0.23511268198490143,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, lactation, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  'token': 22890,
  'token_str': 'lactation'},
 {'score': 0.09662080556154251,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, abortion, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  'token': 17441,
  'token_str': 'abortion'},
 {'score': 0.08252011239528656,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, ovulation, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  

In [16]:
#AYGESTIN (norethindrone acetate tablets USP) is indicated for the treatment of\ 
#secondary amenorrhea, [endometriosis], and abnormal uterine bleeding due to \
#hormonal imbalance in the absence of organic pathology, such as submucous 
#fibroids or uterine cancer.

fill_mask_regular("AYGESTIN (norethindrone acetate tablets USP) is indicated for the \
treatment of secondary amenorrhea, [MASK], and abnormal uterine bleeding due to\
 hormonal imbalance in the absence of organic pathology, such as submucous \
 fibroids or uterine cancer.")

[{'score': 0.14431177079677582,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, pregnancy, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  'token': 5564,
  'token_str': 'pregnancy'},
 {'score': 0.10275698453187943,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, pain, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  'token': 2675,
  'token_str': 'pain'},
 {'score': 0.08248776197433472,
  'sequence': 'aygestin ( norethindrone acetate tablets usp ) is indicated for the treatment of secondary amenorrhea, hysterectomy, and abnormal uterine bleeding due to hormonal imbalance in the absence of organic pathology, such as submucous fibroids or uterine cancer.',
  'token'

In [ ]:
import pickle
import os
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
        

In [ ]:
data_text_features = load_obj("/content/drive/MyDrive/DDI/test_features/data_text_features.pkl")
id_list = load_obj("/content/drive/MyDrive/DDI/test_features/id_list.pkl")
id_list = [int(_) for _ in id_list]
test_id = load_obj("/content/drive/MyDrive/DDI/test_features/test_id_w_ground_truth.pkl")

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def mean_pooling_2(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, [0,1])
    sum_mask = torch.clamp(input_mask_expanded.sum([0,1]), min=1e-9)
    return sum_embeddings.unsqueeze(0) / sum_mask.unsqueeze(0)

In [ ]:
#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog when it is raining.']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModel.from_pretrained("/content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_SDU/")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/DDI/pharma_bert_training/pharma_bert_SDU/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog when it is raining.']
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=256, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

In [ ]:
sentence_embeddings.shape

torch.Size([3, 768])

In [ ]:
for sl in range(0,len(test_id),100):
  if sl <= 0:
    continue
  bert_embedding=[]
  bert_embedding_2 = []
  for i in range(sl,min(sl+100,len(test_id))):
      data_list = []
      data_list_2 = []
      data = data_text_features[id_list.index(test_id[i])]
      print(i)
      
      for j in range(2,len(data)):
          if j == 4:

              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              
              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j ==5:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)

              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j == 6:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 1000:
                  strip_index = 0
                  strip_count = 0
                  for si in range(len(striped_list)):
                      strip_count += len(striped_list[si].split())
                      if strip_count < 1000:
                          strip_index = si 
                      else:
                          break
                  encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
              else:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1].mean(0,keepdim=True))
              continue
          else:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1])
              continue
      bert_embedding.append(np.array(torch.cat(data_list,0)))
      bert_embedding_2.append(np.array(torch.cat(data_list_2,0)))
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/pharmbert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/pharmbert_2/"+str(savedex),bert_np_2)

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
